In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from datetime import date
from datetime import datetime
import calendar
import os
import re


from scipy import stats
from sklearn.neighbors import KernelDensity
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, PillowWriter

import seaborn as sns
from pathlib import Path
from tqdm import tqdm_notebook, trange, tqdm
from PIL import Image, ImageDraw
import io
import random as r

from matplotlib.widgets import Slider, Button, RadioButtons
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
# For downsampling
from sklearn.utils import resample,shuffle

In [3]:
df_companies = pd.read_csv("./data/companies.csv")
df_payments = pd.read_csv("./data/payments.csv")
df_physicians = pd.read_csv("./data/physicians.csv")

In [4]:
df_companies.rename({"State":"State_Comp","Country":"Country_Comp"}, axis='columns', inplace=True)
#df_companies.shape
df_companies

,Company_ID,Name,State_Comp,Country_Comp
0,1,Merck Sharp & Dohme Corporation,NJ,United States
1,2,"COMSORT, Inc",MD,United States
2,3,Daiichi Sankyo Company LTD,NaN,Japan
3,4,"C. R. Bard, Inc. & Subsidiaries",NJ,United States
4,5,Novartis Pharma AG,NaN,Switzerland
...,...,...,...,...
2426,2427,Amryt Pharma Holdings Ltd,NaN,Ireland
2427,2428,"Heraeus Medical Components, LLC",MN,United States
2428,2429,International Life Sciences,GA,United States
2429,2430,"Foundation Consumer Healthcare, LLC",PA,United States


In [5]:
df_physicians.rename({"State":"State_Phys","Country":"Country_Phys"}, axis='columns', inplace=True)
df_physicians

,id,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State_Phys,Zipcode,Country_Phys,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5
0,1,train,LEIGH,B,HOPPE,NaN,FORT PIERCE,FL,34950,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Obstetrics...,FL,NaN,NaN,NaN,NaN
1,2,train,STEVEN,NaN,FRANK,NaN,TOWSON,MD,21204-6808,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Anesthesio...,MD,NaN,NaN,NaN,NaN
2,3,train,THOMAS,NaN,COUCH,JR.,TROY,NY,12180-2832,UNITED STATES,NaN,Podiatric Medicine & Surgery Service Providers...,NY,NaN,NaN,NaN,NaN
3,4,train,CLAUDE,NaN,SU,NaN,SAVANNAH,GA,31419-1753,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Internal M...,GA,SC,AL,NaN,NaN
4,5,train,ROBERT,C,SCHENCK,NaN,ALBUQUERQUE,NM,87131-0001,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Orthopaedi...,NM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5996,train,MARC,NaN,WEINSTEIN,NaN,TEMPLE TERRACE,FL,33637-0925,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Orthopaedi...,FL,NaN,NaN,NaN,NaN
5996,5997,train,CHARLES,W,YOWELL,NaN,TALLAHASSEE,FL,32308-4620,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Urology,FL,SC,NaN,NaN,NaN
5997,5998,train,MARK,O,GABBIE,NaN,TEXARKANA,AR,71854,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Family Med...,AR,TX,NaN,NaN,NaN
5998,5999,test,KRISHNASWAMY,NaN,GAJARAJ,NaN,RANDOLPH,MA,02368-2100,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,MA,NaN,NaN,NaN,NaN


In [11]:
pd.set_option('display.max_columns', None)
df_payments.head()
#df_payments.shape

(1402250, 29)

In [9]:
df_payments.describe(include="all")

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3
count,1.402250e+06,1.402250e+06,1.402250e+06,1.402250e+06,1402250,1.402250e+06,1402250,1402250,75125,72804,75128,1402250,1402250,767529,19532,156984,1402250,1017616,260063,83564,1316057,299888,102499,1314341,299917,102540,814088,177608,65370
unique,NaN,NaN,NaN,NaN,2344,NaN,7,14,4557,52,62,2,3,2,2,31927,6,2491,1229,843,6,6,6,8919,3077,2019,1678,594,407
top,NaN,NaN,NaN,NaN,11/13/2018,NaN,In-kind items and services,Food and Beverage,Chicago,CA,United States,No,No Third Party Payment,No,No,Informational Meal,Yes,50458-580-30,0169-2660-15,0006-0335-30,Drug,Drug,Drug,XARELTO,Repatha,JANUMET XR,Diabetes,Diabetes,Diabetes
freq,NaN,NaN,NaN,NaN,1212,NaN,1170221,1230887,2328,9651,72804,1399846,1382708,767502,17051,54696,835032,22921,10185,4156,564025,129896,45264,23642,13844,4524,44380,23535,8669
mean,7.011255e+05,3.005276e+03,3.881313e+02,1.811990e+02,NaN,1.024069e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,4.047949e+05,1.740790e+03,4.760497e+02,1.057714e+04,NaN,7.780750e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,3.505632e+05,1.474000e+03,1.140000e+02,1.118000e+01,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,7.011255e+05,3.037000e+03,1.970000e+02,1.521000e+01,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.051688e+06,4.528000e+03,4.730000e+02,2.334000e+01,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#combine into one df
df = pd.merge(df_payments, df_physicians, left_on='Physician_ID', right_on='id', how='left').drop('id', axis=1)
df = pd.merge(df, df_companies, on='Company_ID', how='left')
df.shape


(1402250, 48)

In [7]:

df.rename({"Name":"Company_Name"}, axis='columns', inplace=True)
df

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,...,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,Company_Name,State_Comp,Country_Comp
0,1,2455,597,48.78,08/01/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,...,NaN,Allopathic & Osteopathic Physicians|Internal M...,GA,NaN,NaN,NaN,NaN,"Medtronic Vascular, Inc.",MN,United States
1,2,1534,593,122.43,08/01/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,...,NaN,Allopathic & Osteopathic Physicians|Internal M...,NY,NaN,NaN,NaN,NaN,Amarin Pharma Inc.,NJ,United States
2,3,2632,760,10.26,08/01/2013,1,In-kind items and services,Education,NaN,NaN,...,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,OH,NaN,NaN,NaN,NaN,Questcor Pharmaceuticals,CA,United States
3,4,550,760,13.68,08/01/2013,1,In-kind items and services,Education,NaN,NaN,...,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,CA,NaN,NaN,NaN,NaN,Questcor Pharmaceuticals,CA,United States
4,5,1710,826,17.17,08/01/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,...,NaN,NaN,TN,NaN,NaN,NaN,NaN,Actavis Pharma Inc,NJ,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402245,1402246,4705,401,11838.32,12/31/2019,2,Cash or cash equivalent,Royalty or License,NaN,NaN,...,NaN,Allopathic & Osteopathic Physicians|Neurologic...,FL,NaN,NaN,NaN,NaN,"Globus Medical, Inc.",PA,United States
1402246,1402247,1040,1268,9.57,12/31/2019,1,Cash or cash equivalent,Food and Beverage,NaN,NaN,...,NaN,Allopathic & Osteopathic Physicians|Family Med...,FL,NaN,NaN,NaN,NaN,"Romark Laboratories, LC",FL,United States
1402247,1402248,724,1220,1536.61,12/31/2019,1,Cash or cash equivalent,Travel and Lodging,Vancouver,NaN,...,NaN,Allopathic & Osteopathic Physicians|Orthopaedi...,CA,TX,NaN,NaN,NaN,Spineart USA Inc,CA,United States
1402248,1402249,4495,964,2150.00,12/31/2019,1,Cash or cash equivalent,Compensation for services other than consultin...,NaN,NaN,...,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,CA,PA,NaN,NaN,NaN,"Acorda Therapeutics, Inc",NY,United States


In [12]:
df.columns
print(df.shape)
first_col = df.pop("Ownership_Indicator")
print(df.shape)
df.insert(47,"Ownership_Indicator",first_col)
print(df.shape)
print(df.columns)

(1402250, 48)
(1402250, 47)
(1402250, 48)
Index(['Record_ID', 'Physician_ID', 'Company_ID',
       'Total_Amount_of_Payment_USDollars', 'Date', 'Number_of_Payments',
       'Form_of_Payment_or_Transfer_of_Value',
       'Nature_of_Payment_or_Transfer_of_Value', 'City_of_Travel',
       'State_of_Travel', 'Country_of_Travel', 'Third_Party_Recipient',
       'Charity', 'Third_Party_Covered', 'Contextual_Information',
       'Related_Product_Indicator', 'Product_Code_1', 'Product_Code_2',
       'Product_Code_3', 'Product_Type_1', 'Product_Type_2', 'Product_Type_3',
       'Product_Name_1', 'Product_Name_2', 'Product_Name_3',
       'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'set',
       'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'City',
       'State_Phys', 'Zipcode', 'Country_Phys', 'Province',
       'Primary_Specialty', 'License_State_1', 'License_State_2',
       'License_State_3', 'License_State_4', 'License_State_5', 'Company_Name',
       'Sta

In [13]:
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

In [14]:
df_train['Ownership_Indicator']=np.where((df_train.Ownership_Indicator == "Yes"),1,df_train.Ownership_Indicator)
df_train['Ownership_Indicator']=np.where((df_train.Ownership_Indicator == "No"),0,df_train.Ownership_Indicator)

<ipython-input-14-5ef3ec0b706a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Ownership_Indicator']=np.where((df_train.Ownership_Indicator == "Yes"),1,df_train.Ownership_Indicator)
<ipython-input-14-5ef3ec0b706a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Ownership_Indicator']=np.where((df_train.Ownership_Indicator == "No"),0,df_train.Ownership_Indicator)


In [16]:
df_train.Ownership_Indicator.value_counts()


0    1167626
1       2404
Name: Ownership_Indicator, dtype: int64

In [66]:
df_train_maj=df_train[df_train.Ownership_Indicator==0]
df_train_min=df_train[df_train.Ownership_Indicator==1]
df_train_maj_down=resample(df_train_maj, replace=False, n_samples=2404, random_state=2021)
df_train_final = pd.concat([df_train_maj_down, df_train_min])


In [ ]:
df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
col_list = df_train_final.columns.tolist()
reordered = reorder_columns(col_list, last_cols=['Ownership_Indicator'])
df_train_final=df_train_final[reordered]
df_train_final

In [ ]:

train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
train = shuffle(train)
test = shuffle(test)
features=train[train.columns[0:48]]
y=train['Ownership_Indicator']
x_test = test[test.columns[0:48]]
y_test = test['Ownership_Indicator']